In [3]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import scipy.stats as st
from matplotlib.ticker import FuncFormatter
import scipy.optimize as op
plt.style.use("dark_background") # Config plots for dark mode, delete if on light mode
plt.rcParams['figure.dpi'] = 150 # Hi-res plots

In [4]:
station_data = pd.read_csv("../data/santander_locations.csv")


class OptimizationError(RuntimeError):
    """Called when optimizer does not converge."""
    pass

class StationIdError(IndexError):
    """Called when we try and read a non-existing station id."""
    pass


def get_station_name(in_id):
    """Get station name from bike_data for a given id."""
    try:
        return station_data[
            station_data["Station.Id"] == in_id].StationName.iloc[0]
    except IndexError:
        StationIdError("No station matching input ID")


bike_data = pd.read_csv("../data/processed_df.csv", index_col=0)
x = bike_data.min()["start_time"]
t_min = (x // 86400) * 86400
bike_data["start_time"] = (bike_data["start_time"] - t_min) / 60
bike_data["end_time"] = (bike_data["end_time"] - t_min) / 60
bike_data["start_time"] = bike_data["start_time"] \
    + np.random.rand(*bike_data["start_time"].shape)
bike_data["end_time"] = bike_data["end_time"] \
    + np.random.rand(*bike_data["end_time"].shape)
bike_data["duration"] = bike_data.end_time - bike_data.start_time
bike_data = bike_data.sort_values(by=["start_time"])

train_time = 12*7*24*60
train_bike_data = bike_data[bike_data.start_time <= train_time]
test_bike_data = bike_data[bike_data.start_time > train_time]
train_sorted_stations_start = []
for st_id in train_bike_data.start_id.sort_values().unique():
    train_sorted_stations_start.append(
        train_bike_data[train_bike_data.start_id == st_id]
        )
test_sorted_stations = []
for st_id in test_bike_data.start_id.sort_values().unique():
    test_sorted_stations.append(
        test_bike_data[test_bike_data.start_id == st_id]
        )
rates_dict = {}
for station in test_sorted_stations:
    time_elapsed = station.start_time.to_numpy()[-1] \
        - station.start_time.to_numpy()[0]
    n_events = test_sorted_stations[0].size
    rate = n_events / time_elapsed

    rates_dict[station.start_id.unique()[0]] = rate
station_array = list(rates_dict.keys())


def ecdf(data):
    # https://cmdlinetips.com/2019/05/empirical-cumulative-distribution-function-ecdf-in-python/
    """ Compute ECDF """
    x = np.sort(data)
    n = x.size
    y = np.arange(1, n+1) / n
    return(x, y)


tprime_per_station = {}
for id in bike_data.end_id.unique():
    unsorted_station_end_time = bike_data[bike_data.end_id == id]
    sorted_station_end_time = unsorted_station_end_time.sort_values(
        by=["end_time"])
    tprime_per_station[id] = sorted_station_end_time.\
        end_time.to_numpy()
tprime_per_station

t_per_station = {}
for id in bike_data.start_id.unique():
    unsorted_station_start_time = bike_data[bike_data.start_id == id]
    sorted_station_start_time = unsorted_station_start_time.sort_values(
        by=["start_time"])
    t_per_station[id] = sorted_station_start_time.\
        start_time.to_numpy()

sorted_start_ids = np.sort(bike_data.start_id.unique())

In [5]:
def N(t_scalar, t):
    """
    Returns the number of times in t less than or equal to t_scalar.
    Is used to compute N(t_{i,k}) and N'(t_{i,k}) depending on whether t above is t or t_prime
    """

    return np.searchsorted(t, t_scalar, side="right")


def B(h, t, t_prime, beta):

    """
    Returns a list of [B_i(1), ..., B_i(h)]

    NOTE: t_prime NEEDS to be sorted here

    Note all index variables such as h, k, etc start at 1, like the mathematical notation.
    """
    B = []

    # Append base case B_i(1)
    B.append(np.sum([np.exp(-1*beta*(t[0] - t_prime[k-1])) for k in range(1, N(t[0], t_prime) + 1)]))

    # Append the rest
    for l in range(2, h+1):

        # First term in recursive formula for B_i(h)
        term1 = np.exp(-1*beta*(t[l-1] - t[l-2])) * B[l-2]

        # Second term
        #term2 = np.sum([np.exp(-1*beta*(t[l-1] - t_prime[k-1])) for k in range(N(t[l-2], t_prime) + 1, N(t[l-1], t_prime) + 1)])

        lower = N(t[l-2], t_prime)
        upper = N(t[l-1], t_prime)

        term2 = np.sum(np.exp(-1*beta*(t[l-1] - t_prime[lower:upper]))) # IMP: This is the term taking the most time by far

        B.append(term1 + term2)

    return B


def compensator_m3(t_scalar, t_prime, lambda_i, alpha_i, beta_i):
    """
    t_scalar: scalar value where Lambda_i(t) is to be evaluated
    t_prime: list of arrival times at station i

    NOTE: t_prime NEEDS TO BE SORTED HERE.
    """

    term1 = lambda_i * t_scalar
    term2 = -(alpha_i / beta_i) * np.sum([np.exp(-beta_i * (t_scalar - t_prime[k-1])) - 1 for k in range(1, N(t_scalar, t_prime) + 1)])

    return term1 + term2


In [6]:
def m3_log_likelihood(t, t_prime, alpha_i, beta_i, lambda_i):

    """
    Gives log likelihood of our three parameters. 
    t: start times from station i
    t_prime: end times at station i

    NOTE: t_prime NEEDS TO BE SORTED HERE
    """
    
    T = t[-1] # TODO: Is this how we get big T?

    # Get B list 
    B_ = np.array(B(len(t), t, t_prime, beta_i))

    #term1 = np.sum(np.log(lambda_i + alpha_i*B_))
    term1 = np.sum([np.log(lambda_i + alpha_i*B_[j-1]) for j in range(1, len(t) + 1)])

    term2 = -1 * compensator_m3(T, t_prime, lambda_i, alpha_i, beta_i)

    return term1 + term2



In [7]:
def getTimeDifferences(t, t_prime):
    """
    Input: (sorted) times for a particular station i
    Output: List of differences indexed by [h][k] for this station i
    """

    # h goes until N(t[-1], t) assuming T = t[-1]
    T = t[-1]
    D_result = []
    for h in range(1, N(T, t)+1):
        differences_list = []
        # Construct list of t_ih - t'_ik for k = 1 to N'(T)
        for k in range(N(t[h-2], t_prime) + 1, N(t[h-1], t_prime) + 1):
            differences_list.append(t[h-1] - t_prime[k-1])

        D_result.append(np.array(differences_list))

    return D_result

In [11]:
def new_B(h, t, t_prime, beta, time_differences):

    """
    Returns a list of [B_i(1), ..., B_i(h)]

    NOTE: t_prime NEEDS to be sorted here

    time_differences: time differences double list for station i

    Note all index variables such as h, k, etc start at 1, like the mathematical notation.
    """
    B = []

    # Append base case B_i(1)
    B.append(np.sum([np.exp(-1*beta*(t[0] - t_prime[k-1])) for k in range(1, N(t[0], t_prime) + 1)]))

    # Append the rest
    for l in range(2, h+1):

        # First term in recursive formula for B_i(h)
        term1 = np.exp(-1*beta*(t[l-1] - t[l-2])) * B[l-2]

        term2 = np.sum(np.exp(-1*beta*(time_differences[l-1])))
        B.append(term1 + term2)

    return B

def new_m3_log_likelihood(t, t_prime, alpha_i, beta_i, lambda_i, time_differences):
    """
    Gives log likelihood of our three parameters. 
    t: start times from station i
    t_prime: end times at station i

    NOTE: t_prime NEEDS TO BE SORTED HERE
    """
    
    T = t[-1] # TODO: Is this how we get big T?

    # Get B list 
    B_ = np.array(new_B(len(t), t, t_prime, beta_i, time_differences))

    #term1 = np.sum(np.log(lambda_i + alpha_i*B_))
    term1 = np.sum([np.log(lambda_i + alpha_i*B_[j-1]) for j in range(1, len(t) + 1)])

    term2 = -1 * compensator_m3(T, t_prime, lambda_i, alpha_i, beta_i)

    return term1 + term2


## Test whether our B function with time differences previously computed works

In [267]:
beta = 0.01

t = t_per_station[1]
t_prime = tprime_per_station[1]

time_differences = getTimeDifferences(t, t_prime) # TODO: convert ragged list to numpy? Elements inside list have different lengths
#time_differences
print(B(9, t, t_prime, beta))
print(new_B(9, t, t_prime, beta, time_differences))

[0.0, 0.0, 1.5703055024796566, 1.5477079539237808, 2.1467989487940153, 1.9292432014786471, 3.0222128387798928, 2.6028363838903075, 2.7420225109216565]
[0.0, 0.0, 1.5703055024796566, 1.5477079539237808, 2.1467989487940153, 1.9292432014786471, 3.0222128387798928, 2.6028363838903075, 2.7420225109216565]


In [273]:
t = t_per_station[1]
t_prime = tprime_per_station[1]

time_differences = getTimeDifferences(t, t_prime)
print(new_m3_log_likelihood(t, t_prime, 0.1, 1, 0.1, time_differences))
print(m3_log_likelihood(t, t_prime, 0.1, 1, 0.1))


-21360.786147124014
-21360.786147124014


## Finding the parameters using likelihood optimisation

In [12]:
optimal_parameters = {}
for st_id in sorted_start_ids:
    print(st_id)
    x0 = [np.log(0.01), np.log(0.05), np.log(0.1)] # np.log(rates_dict[station.start_id.unique()[0]])]

    # TODO: What bounds should we use here?
    t = t_per_station[st_id]
    t_prime = tprime_per_station[st_id] # Need to sort t_prime for likelihood function
    time_differences = getTimeDifferences(t, t_prime)

    op_m3_likelihood = lambda x: -new_m3_log_likelihood(t, t_prime, np.exp(x[0]), np.exp(x[0]) + np.exp(x[1]), np.exp(x[2]), time_differences)
    sol = op.minimize(op_m3_likelihood, x0, method="Nelder-Mead")

    #sol = op.minimize(op_m3_likelihood, x0, method="SLSQP")
    if sol.success:
        transformed_alpha = np.exp(sol.x[0])
        transformed_beta = np.exp(sol.x[1]) + np.exp(sol.x[0])
        transformed_lambda = np.exp(sol.x[2])
        max_params = [transformed_alpha, transformed_beta, transformed_lambda]
        optimal_parameters[st_id] = max_params

    else:
        raise OptimizationError(f"Failed to converge for station {station}.")
optimal_parameters

1
2
3
4
5
6


KeyboardInterrupt: 

In [276]:
optimal_parameters = {}
for station in train_sorted_stations_start:
    print(station.start_id.to_numpy()[0])
    x0 = [0.1, 1, 0.1] # np.log(rates_dict[station.start_id.unique()[0]])]

    t = station.start_time.to_numpy()
    t_prime = np.sort(station.end_time.to_numpy()) # Need to sort t_prime for likelihood function
    time_differences = getTimeDifferences(t, t_prime)

    op_m3_likelihood = lambda x: -new_m3_log_likelihood(t, t_prime, x[0], x[1], x[2], time_differences)
    bounds = ((0.0000001, 10), (0.0000001, 10), (0.0000001, 10))
    #sol = op.minimize(op_m3_likelihood, x0, method="Nelder-Mead", bounds=bounds)
    sol = op.minimize(op_m3_likelihood, x0, method="Nelder-Mead", bounds=bounds)
    if sol.success:
        max_params = sol.x
        optimal_parameters[station.start_id.unique()[0]] = max_params

    else:
        raise OptimizationError(f"Failed to converge.")
optimal_parameters

1
2
3


KeyboardInterrupt: 

## Assessing fit for model 3